In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import re
import json
import ast
from nltk.tokenize import word_tokenize
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unlocking-arabic-language-comprehension-with-the/validation.csv
/kaggle/input/unlocking-arabic-language-comprehension-with-the/train.csv


In [25]:
import re

In [26]:
data = pd.read_csv("/kaggle/input/unlocking-arabic-language-comprehension-with-the/train.csv")

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        693 non-null    int64 
 1   title     693 non-null    object
 2   context   693 non-null    object
 3   question  693 non-null    object
 4   answers   693 non-null    object
dtypes: int64(1), object(4)
memory usage: 27.2+ KB


In [28]:
data.head(20)

,id,title,context,question,answers
0,969331847966,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- من هو جمال أحمد حمزة خاشقجي؟,"{'text': array(['صحفي وإعلامي'], dtype=object)..."
1,115150665555,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,{'text': array(['حمزة خاشقجي (13 أكتوبر 1958، ...
2,74212080718,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,"{'text': array(['المدينة المنورة'], dtype=obje..."
3,465699296586,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,"{'text': array(['واشنطن بوست'], dtype=object),..."
4,564177542570,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,{'text': array(['وُصف في الصحف وأجهزة الاعلام ...
5,98945395462,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي منصب شغل في الجريدة؟ ال,"{'text': array(['وتقلّد منصب مستشار،'], dtype=..."
6,838329318053,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- متى غادر خاشقجي السعودية؟ ال,"{'text': array(['في سبتمبر 2017،'], dtype=obje..."
7,432910739583,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- لمن ينتقده في مقالاته الإخبارية؟ ال,"{'text': array(['الحكومة السعودية.'], dtype=ob..."
8,619452973730,جمال خاشقجي,غادر خاشقجي السعودية في سبتمبر 2017، وكتب بعد ...,- لمن انتقد خاشقجي بشدة في مقالاته الإخبارية؟ ال,{'text': array(['ولي العهد السعودي محمد بن سلم...
9,333772766499,السعودية,السعودية أو (رسميًا: المملكة العربية السعودية)...,- أي دولة هي أكبر دولة في الشرق الأوسط؟ ال,"{'text': array(['المملكة العربية السعودية)'], ..."


In [29]:
def normalize_arabic(text):
    text = re.sub(r'[^\w\s]', '', text) 
    text = text.lstrip()
    arabic_diacritics = re.compile(
        r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(arabic_diacritics, '', text) 
    return text

In [30]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('arabic'))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [31]:
from nltk.stem.isri import ISRIStemmer


def stemmer(tokens):
    stemmer = ISRIStemmer()

    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

In [32]:
text = data.context[1]
print(text)

جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا.


In [33]:
text = normalize_arabic(text)
print(text)

جمال أحمد حمزة خاشقجي 13 أكتوبر 1958 المدينة المنورة  2 أكتوبر 2018 صحفي وإعلامي سعودي رأس عدة مناصب لعدد من الصحف في السعودية وتقلد منصب مستشار كما أنه مدير عام قناة العرب الإخبارية سابقا


In [34]:
def preprocess_text(text):
    text = normalize_arabic(text)
    text =  remove_stopwords(text)
    text = stemmer(text)
    return text

In [35]:
data["cleaned_context"] = data["context"].apply(lambda text:preprocess_text(text))
data["cleaned_question"] = data["question"].apply(lambda text:preprocess_text(text))

In [36]:
data["anwers_text"] = data["answers"].str.extract(r"text':\s*array\(\[(.*?)\],\s*dtype=object\)")

In [37]:
data["answer_start"] = data["answers"].str.extract(r"answer_start':\s*array\(\[(.*?)\],\s*dtype=int32\)")

In [38]:
data.head()

,id,title,context,question,answers,cleaned_context,cleaned_question,anwers_text,answer_start
0,969331847966,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- من هو جمال أحمد حمزة خاشقجي؟,"{'text': array(['صحفي وإعلامي'], dtype=object)...","[جمل, حمد, حمز, خاشقج, 13, 1958, دين, نور, 2, ...","[جمل, حمد, حمز, خاشقج]",'صحفي وإعلامي',73
1,115150665555,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال,{'text': array(['حمزة خاشقجي (13 أكتوبر 1958، ...,"[جمل, حمد, حمز, خاشقج, 13, 1958, دين, نور, 2, ...","[ولد, جمل, حمد, حمز, خاشقج, وتف, ال]",'حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة ...,10
2,74212080718,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي مدينة ولد جمال أحمد حمزة خاشقجي؟ ال,"{'text': array(['المدينة المنورة'], dtype=obje...","[جمل, حمد, حمز, خاشقج, 13, 1958, دين, نور, 2, ...","[دين, ولد, جمل, حمد, حمز, خاشقج, ال]",'المدينة المنورة',39
3,465699296586,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- في أي صحيفة قام بكتابة عمود منذ عام 2017؟ ال,"{'text': array(['واشنطن بوست'], dtype=object),...","[جمل, حمد, حمز, خاشقج, 13, 1958, دين, نور, 2, ...","[صحف, كتب, عمد, عام, 2017, ال]",'واشنطن بوست',224
4,564177542570,جمال خاشقجي,جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة...,- كيف وصفها في الصحف ووسائل الإعلام الدولية؟ ال,{'text': array(['وُصف في الصحف وأجهزة الاعلام ...,"[جمل, حمد, حمز, خاشقج, 13, 1958, دين, نور, 2, ...","[وصف, صحف, وسل, علم, دول, ال]","'وُصف في الصحف وأجهزة الاعلام العالمية بأنه ""و...",246


In [39]:
unique_context = data.context.unique()

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(data.context)

TfidfVectorizer()

In [41]:
data["context_question"] = data["context"]+" "+data["question"]

In [42]:
X = vectorizer.transform(data["context_question"])
y = vectorizer.transform(data["anwers_text"])
y = y.toarray()

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [45]:
from sklearn.multioutput import MultiOutputRegressor

multi_output_regressor = MultiOutputRegressor(LinearRegression())

y_train = np.vstack(y_train)
multi_output_regressor.fit(X_train, y_train)

y_pred = multi_output_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.008694500422621885


In [46]:
from sklearn.metrics.pairwise import cosine_similarity
answer_tfidf = vectorizer.transform(data['anwers_text'])
answer_tfidf = answer_tfidf.toarray()

def retrieve_answer(question, context):
    text = question + ' ' + context
    text_tfidf = vectorizer.transform([text])
    
    predicted_tfidf = multi_output_regressor.predict(text_tfidf)[0]
    
    predicted_tfidf = np.array([predicted_tfidf])
    
    similarities = cosine_similarity(predicted_tfidf, answer_tfidf)
    best_index = np.argmax(similarities)
    answer = data['answers'].iloc[best_index]

    return answer

#  usagedata["anwers_text"] = data["answers"].str
question = data["question"][13]
context = data["context"][13]
print(question)
print(data["answers"][13])
print("Retrieved Answer:", retrieve_answer(question, context))


- ما هي الحدود في الشمال الشرقي؟ ال
{'text': array(['الكويت'], dtype=object), 'answer_start': array([237], dtype=int32)}
Retrieved Answer: {'text': array(['الكويت'], dtype=object), 'answer_start': array([237], dtype=int32)}


In [47]:
question = data["question"][1]
context = data["context"][1]

In [48]:
print(question)

- متى ولد جمال أحمد حمزة خاشقجي وتوفي؟ ال


In [49]:
print(context)

جمال أحمد حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)، صحفي وإعلامي سعودي، رأس عدّة مناصب لعدد من الصحف في السعودية، وتقلّد منصب مستشار، كما أنّه مدير عام قناة العرب الإخبارية سابقًا.


In [50]:
print("Retrieved Answer:", retrieve_answer(question, context))

Retrieved Answer: {'text': array(['حمزة خاشقجي (13 أكتوبر 1958، المدينة المنورة - 2 أكتوبر 2018)،'],
      dtype=object), 'answer_start': array([10], dtype=int32)}
